In [2]:
import numpy as np
import sklearn
from sklearn import preprocessing, neighbors
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from typing import Tuple
from sklearn import model_selection
from sklearn.metrics import classification_report

import pandas as pd
import random, os
import csv

rng = 1

def seed_everything(seed=1):
    """"
    Seed everything.
    """   
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
#print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [3]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import scale

In [4]:
#cluster test/train data
def get_clusters(X_train: pd.DataFrame, X_test: pd.DataFrame, n_clusters: int) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    applies k-means clustering to training data to find clusters and predicts them for the test set
    """
    clustering = KMeans(n_clusters=n_clusters, random_state=rng)
    clustering.fit(X_train)
    # apply the labels to the training set
    train_labels = clustering.labels_
    X_train_clstrs = X_train.copy()
    X_train_clstrs['clusters'] = train_labels
    
    #write ext_clusters to csv
    X_train_clstrs.to_csv('X_train_clstrs.csv')
    
    # predict labels on the test set
    test_labels = clustering.predict(X_test)
    X_test_clstrs = X_test.copy()
    X_test_clstrs['clusters'] = test_labels
    return X_train_clstrs, X_test_clstrs

#scale each feature
def scale_features(X_train: pd.DataFrame, X_test: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    applies standard scaler (z-scores) to training data and predicts z-scores for the test set
    """
    scaler = StandardScaler()
    to_scale = [col for col in X_train.columns.values]
    scaler.fit(X_train[to_scale])
    X_train[to_scale] = scaler.transform(X_train[to_scale])
    
    # predict z-scores on the test set
    X_test[to_scale] = scaler.transform(X_test[to_scale])
    
    return X_train, X_test



In [5]:
df = pd.read_csv('..\..\data\_All_Receptors_runs_1_2_3_binary.csv')
df.drop(['min_feat','receptor','Active_Rate','Enrichment', 'GH', 'Actives', 'filename', 'fbase', 'hyd', 'don', 'acc', 'ani', 'cat', 'aro', 'donhyd', 'catdon', 'hydaro', 'aniacc', 'donacc', 'acc_prop', 'ani_prop', 'cat_prop', 'aro_prop', 'donhyd_prop', 'donacc_prop'], axis=1, inplace=True)
df.fillna(-99999)

predictors = list(df.columns)
predictors = predictors[:-1]

print('Predictors:', predictors,'\n')

Predictors: ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 's_score', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 



In [6]:
#read in train/test data
df = pd.read_csv('..\..\data\_All_Receptors_runs_1_2_3_binary.csv')
df = df[['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 's_score', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop', 'quality']]
df.fillna(-99999)

predictors = list(df.columns)
predictors = predictors[:-1]

print('Predictors:', predictors,'\n')


#split data into quality/not quality sets
q_ph4s = df[df['quality'] == 1]
nq_ph4s = df[df['quality'] != 1]

#ensure that there is an equal number of nq ph4s
nq_ph4s = nq_ph4s.sample(n=2*len(q_ph4s), random_state = rng)
#print(type(nq_ph4s))
#print(nq_ph4s['Hits'].head())
#print(nq_ph4s['Hits'].tail())

#merge arrays prior to TTS
frames = [q_ph4s, nq_ph4s]
df = pd.concat(frames)

#x is features, y is classes
x = df.drop('quality', axis=1)
y = df.quality

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=rng)

print("x_train Q ph4s:", y_train.sum(),'\n')
print("x_test Q ph4s:",y_test.sum(),'\n')

X_train_clstrs, X_test_clstrs = get_clusters(x_train, x_test, 4)

X_train_scaled, X_test_scaled = scale_features(X_train_clstrs, X_test_clstrs)

#print(X_train_scaled)

# to divide the df by cluster, we need to ensure we use the correct class labels, we'll use pandas to do that
train_clusters = X_train_scaled.copy()
test_clusters = X_test_scaled.copy()
train_clusters['y'] = y_train
test_clusters['y'] = y_test

uniq_clusters = train_clusters['clusters'].unique()
uniqs = uniq_clusters.tolist()
uniqs.sort()
print(uniqs)

#print(y_train)
#print(train_clusters['clusters'])

#print(type(clusters))
#print(type(train_clusters['clusters']))
#frames = [clusters, train_clusters['clusters']]
#df = pd.concat(frames, axis=1)
#print(type(df))
#print(df)
#df.to_csv('4clusters.csv')

# locate the "0" cluster
train_0 = train_clusters.loc[train_clusters.clusters <= uniqs[0]] # after scaling, 0 went to -2.187
test_0 = test_clusters.loc[test_clusters.clusters <= uniqs[0]]
y_train_0 = train_0.y.values
y_test_0 = test_0.y.values
# locate the "1" cluster
train_1 = train_clusters.loc[(train_clusters.clusters <= uniqs[1]) & (train_clusters.clusters > uniqs[0])] # after scaling, 1 went to -0.62
test_1 = test_clusters.loc[(test_clusters.clusters <= uniqs[1]) & (test_clusters.clusters > uniqs[0])]
y_train_1 = train_1.y.values
y_test_1 = test_1.y.values

# locate the "2" cluster
train_2 = train_clusters.loc[(train_clusters.clusters <= uniqs[2]) & (train_clusters.clusters > uniqs[1])] # after scaling, 2 went to 0.945
test_2 = test_clusters.loc[(test_clusters.clusters <= uniqs[2]) & (test_clusters.clusters > uniqs[1])]
y_train_2 = train_2.y.values
y_test_2 = test_2.y.values

# locate the "3" cluster
train_3 = train_clusters.loc[train_clusters.clusters >= uniqs[3]] # after scaling, 3 went to 2.51
test_3 = test_clusters.loc[test_clusters.clusters >= uniqs[3]]
y_train_3 = train_3.y.values
y_test_3 = test_3.y.values

# drop the targets from the training set
X_train_0 = train_0.drop(columns=['y'])
X_test_0 = test_0.drop(columns=['y'])
X_train_1 = train_1.drop(columns=['y'])
X_test_1 = test_1.drop(columns=['y'])
X_train_2 = train_2.drop(columns=['y'])
X_test_2 = test_2.drop(columns=['y'])
X_train_3 = train_3.drop(columns=['y'])
X_test_3 = test_3.drop(columns=['y'])

print('X_train 0/1/2/3 cluster values\n')
print('-------------------------------\n')
print(X_train_0['clusters'].unique())
print(X_train_1['clusters'].unique())
print(X_train_2['clusters'].unique())
print(X_train_3['clusters'].unique(),'\n')

#print(X_train_0)
#print(len(X_test_1))


#0 cluster LR model
sgdc0 = SGDClassifier(loss="log", penalty="l1", max_iter=1000, tol=1e-2, class_weight='balanced', random_state = rng)
sgdc0.fit(X_train_0, y_train_0)

y_pred = (sgdc0.predict(X_test_0))
confmat = confusion_matrix(y_test_0, y_pred, labels=[0,1])
#print(confmat)

confmat = confusion_matrix(y_test_0, y_pred, labels=[0,1]).ravel()
FP = (confmat[1])
TP = (confmat[3])

PPV = (TP / (TP + FP))

cm = pd.crosstab(y_test_0, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)

print('0 cluster model\n')
print(cm,'\n')

print('PPV:', format(PPV, '.2f'),'\n')

#1 cluster LR model
sgdc1 = SGDClassifier(loss="log", penalty="l1", max_iter=1000, tol=1e-2, class_weight='balanced', random_state = rng)
sgdc1.fit(X_train_1, y_train_1)

y_pred = (sgdc1.predict(X_test_1))
confmat = confusion_matrix(y_test_1, y_pred, labels=[0,1])

confmat = confusion_matrix(y_test_1, y_pred, labels=[0,1]).ravel()
FP = (confmat[1])
TP = (confmat[3])

PPV = (TP / (TP + FP))

cm = pd.crosstab(y_test_1, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)

print('1 cluster model\n')
print(cm,'\n')

print('PPV:', format(PPV, '.2f'))

#print(X_train_0)
#print(len(X_train_2))

#2 cluster LR model
sgdc2 = SGDClassifier(loss="log", penalty="l1", max_iter=1000, tol=1e-2, class_weight='balanced', random_state = rng)
sgdc2.fit(X_train_2, y_train_2)

y_pred = (sgdc2.predict(X_test_2))
confmat = confusion_matrix(y_test_2, y_pred, labels=[0,1])
#print(confmat)

confmat = confusion_matrix(y_test_2, y_pred, labels=[0,1]).ravel()
FP = (confmat[1])
TP = (confmat[3])

PPV = (TP / (TP + FP))

cm = pd.crosstab(y_test_2, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)

print('2 cluster model\n')
print(cm,'\n')

print('PPV:', format(PPV, '.2f'),'\n')

#3 cluster LR model
sgdc3 = SGDClassifier(loss="log", penalty="l1", max_iter=1000, tol=1e-2, class_weight='balanced', random_state = rng)
sgdc3.fit(X_train_3, y_train_3)

y_pred = (sgdc3.predict(X_test_3))
confmat = confusion_matrix(y_test_3, y_pred, labels=[0,1])
#print(confmat)

confmat = confusion_matrix(y_test_3, y_pred, labels=[0,1]).ravel()
FP = (confmat[1])
TP = (confmat[3])

PPV = (TP / (TP + FP))

cm = pd.crosstab(y_test_3, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)

print('3 cluster model\n')
print(cm,'\n')

print('PPV:', format(PPV, '.2f'))

Predictors: ['Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 's_score', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

x_train Q ph4s: 2699 

x_test Q ph4s: 922 

[-2.261291652098838, -0.6232776848498619, 1.0147362823991142, 2.65275024964809]
X_train 0/1/2/3 cluster values

-------------------------------

[-2.26129165]
[-0.62327768]
[1.01473628]
[2.65275025] 

0 cluster model

Predicted   0   1
Actual           
0          34  14
1          56  54 

PPV: 0.79 

1 cluster model

Predicted    0    1
Actual             
0          577  212
1           79  542 

PPV: 0.72
2 cluster model

Predicted    0    1
Actual             
0          865   80
1            5  169 

PPV: 0.68 

3 cluster model

Predicted  0   1
Actual          
0          5   7
1          1  16 

PPV: 0.70


In [131]:
def scale_features_single(X: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    applies standard scaler (z-scores) to training data and predicts z-scores for the test set
    """
    scaler = StandardScaler()
    to_scale = [col for col in X.columns.values]
    scaler.fit(X[to_scale])
    X[to_scale] = scaler.transform(X[to_scale])
    
    return X
      
def classify_ext_data(input_csv, subset = "none"):      
    ext_df = pd.read_csv(input_csv)
    ext_df.fillna(-99999)
    receptors = ext_df.Receptor
    hits_actual = ext_df.Hits
    score_types = ext_df['Score Type']
    subsets = ext_df.subset
    init_ext_df = ext_df
    
    #check if a 'quality' column exists. one will not exist if classifying data with unknown enrichments.
    if 'quality' not in ext_df:
        ext_df = ext_df[['s_score','Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop']]
        x = ext_df
    else:
        ext_df = ext_df[['s_score','Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop', 'quality']]   
        x = ext_df.drop('quality', axis=1)
        y = ext_df.quality
    
    predictors = list(ext_df.columns)
    predictors = predictors[:-1]
    print('Predictors:', predictors,'\n')
    
    if 'quality' in init_ext_df:
        print("score based Q ph4s:", y.sum(),'\n')
    
    #cluster training data
    clustering = KMeans(n_clusters=4, random_state=rng)
    clustering.fit(x_train)
    # apply the labels to the training set
    train_labels = clustering.labels_
    X_train_clstrs = x_train.copy()
    X_train_clstrs['clusters'] = train_labels
    
    # predict labels on the external set
    ext_labels = clustering.predict(x)
    X_clstrs = x.copy()
    X_clstrs['clusters'] = ext_labels
    
    X_scaled = scale_features_single(X_clstrs)
    ext_clusters = X_scaled.copy()
    if 'quality' in init_ext_df:
        ext_clusters['y'] = y
    
    #add receptors, hits_actual, score type, and subset columns back prior to 0/1/2/3 split
    ext_clusters['Receptor'] = receptors
    ext_clusters['hits_actual'] = hits_actual
    ext_clusters['Score Type'] = score_types
    ext_clusters['subset'] = subsets
    
    #write ext_clusters to csv
    X_train_clstrs.to_csv('X_train_clstrs2.csv')
    
    # locate the "0" cluster
    ext_0 = ext_clusters.loc[ext_clusters.clusters <= uniqs[0]] # after scaling, 0 went negtive
    if 'quality' in init_ext_df:
        y_ext_0 = ext_0.y.values
    ext_0_receptors = ext_0.Receptor
    ext_0_hits_actual = ext_0.hits_actual
    ext_0_score_types = ext_0['Score Type']
    ext_0_subsets = ext_0.subset
    
    # locate the "1" cluster
    ext_1 = ext_clusters.loc[(ext_clusters.clusters <= uniqs[1]) & (ext_clusters.clusters > uniqs[0])] # after scaling, 0 went negtive
    if 'quality' in init_ext_df:
        y_ext_1 = ext_1.y.values
    ext_1_receptors = ext_1.Receptor
    ext_1_hits_actual = ext_1.hits_actual
    ext_1_score_types = ext_1['Score Type']
    ext_1_subsets = ext_1.subset
    
    # locate the "2" cluster
    ext_2 = ext_clusters.loc[(ext_clusters.clusters <= uniqs[2]) & (ext_clusters.clusters > uniqs[1])] # after scaling, 0 went negtive
    if 'quality' in init_ext_df:
        y_ext_2 = ext_2.y.values
    ext_2_receptors = ext_2.Receptor
    ext_2_hits_actual = ext_2.hits_actual
    ext_2_score_types = ext_2['Score Type']
    ext_2_subsets = ext_2.subset
    
    # locate the "3" cluster
    ext_3 = ext_clusters.loc[ext_clusters.clusters > uniqs[3] ] # after scaling, 0 went negtive
    if 'quality' in init_ext_df:
        y_ext_3 = ext_3.y.values
    ext_3_receptors = ext_3.Receptor
    ext_3_hits_actual = ext_3.hits_actual
    ext_3_score_types = ext_3['Score Type']
    ext_3_subsets = ext_3.subset

    # drop the targets from each external set (if classifying known external data)
    if 'quality' in init_ext_df:
        X_ext_0 = ext_0.drop(columns=['y', 'Receptor', 'hits_actual', 'Score Type', 'subset'])
        X_ext_1 = ext_1.drop(columns=['y', 'Receptor', 'hits_actual', 'Score Type', 'subset'])
        X_ext_2 = ext_2.drop(columns=['y', 'Receptor', 'hits_actual', 'Score Type', 'subset'])
        X_ext_3 = ext_3.drop(columns=['y', 'Receptor', 'hits_actual', 'Score Type', 'subset'])
    else:
        X_ext_0 = ext_0.drop(columns=['Receptor', 'hits_actual', 'Score Type', 'subset'])
        X_ext_1 = ext_1.drop(columns=['Receptor', 'hits_actual', 'Score Type', 'subset'])
        X_ext_2 = ext_2.drop(columns=['Receptor', 'hits_actual', 'Score Type', 'subset'])
        X_ext_3 = ext_3.drop(columns=['Receptor', 'hits_actual', 'Score Type', 'subset'])
    
    # drop receptor column from each external set
    
    #print(len(X_ext_0))
    #print(len(X_ext_1))
    #print(len(X_ext_2))
    #print(len(X_ext_3))
    
    #predict based on 0 cluster model
    print('0 cluster model\n')
    print('---------------\n')
    if len(X_ext_0) == 0:
        print('No cluster 0 data.\n')
    else:
        y_pred = (sgdc0.predict(X_ext_0))
        if 'quality' in init_ext_df:
            confmat = confusion_matrix(y_ext_0, y_pred, labels=[0,1])
            confmat = confusion_matrix(y_ext_0, y_pred, labels=[0,1]).ravel()
            FP = (confmat[1])
            TP = (confmat[3])
            PPV = (TP / (TP + FP))
            cm = pd.crosstab(y_ext_0, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)
            print(cm,'\n')
            print('PPV:', format(PPV, '.2f'),'\n')
        
        X_ext_0['Receptor'] = ext_0_receptors
        X_ext_0['hits_actual'] = ext_0_hits_actual
        X_ext_0['Score Type'] = ext_0_score_types
        X_ext_0['subset'] = ext_0_subsets
        if 'quality' in init_ext_df:
            X_ext_0['quality'] = y_ext_0
        
        X_ext_0['quality_pred'] = y_pred
        if 'quality' in init_ext_df:
            X_ext_0.to_csv('results/'+subset+'/0cluster_results.csv')
        #if classifying unknown external data, print the ph4s classified as quality and write them to .csv
        if 'quality' not in init_ext_df:
            print(X_ext_0.loc[X_ext_0['quality_pred'] == 1], '\n')
            ph4_preds = X_ext_0.loc[X_ext_0['quality_pred'] == 1]
            ph4_preds.to_csv('0cluster_ph4_preds.csv')

    print('1 cluster model\n')
    print('---------------\n')
    if len(X_ext_1) == 0:
        print('No cluster 1 data.\n')
    else:
        y_pred = (sgdc1.predict(X_ext_1))
        if 'quality' in init_ext_df:
            confmat = confusion_matrix(y_ext_1, y_pred, labels=[0,1])
            confmat = confusion_matrix(y_ext_1, y_pred, labels=[0,1]).ravel()
            FP = (confmat[1])
            TP = (confmat[3])
            PPV = (TP / (TP + FP))
            cm = pd.crosstab(y_ext_1, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)
            print(cm,'\n')
            print('PPV:', format(PPV, '.2f'),'\n')
        
        X_ext_1['Receptor'] = ext_1_receptors
        X_ext_1['hits_actual'] = ext_1_hits_actual
        X_ext_1['Score Type'] = ext_1_score_types
        X_ext_1['subset'] = ext_1_subsets
        if 'quality' in init_ext_df:
            X_ext_1['quality'] = y_ext_1
        
        X_ext_1['quality_pred'] = y_pred
        if 'quality' in init_ext_df:
            X_ext_1.to_csv('results/'+subset+'/1cluster_results.csv')
            
        if 'quality' not in init_ext_df:
            print(X_ext_1.loc[X_ext_1['quality_pred'] == 1], '\n')
            ph4_preds = X_ext_1.loc[X_ext_1['quality_pred'] == 1]
            ph4_preds.to_csv('1cluster_ph4_preds.csv')
        
    print('2 cluster model\n')
    print('---------------\n')
    if len(X_ext_2) == 0:
        print('No cluster 2 data.\n')
    else:
        y_pred = (sgdc2.predict(X_ext_2))
        if 'quality' in init_ext_df:
            confmat = confusion_matrix(y_ext_2, y_pred, labels=[0,1])
            confmat = confusion_matrix(y_ext_2, y_pred, labels=[0,1]).ravel()
            FP = (confmat[1])
            TP = (confmat[3])
            PPV = (TP / (TP + FP))
            cm = pd.crosstab(y_ext_2, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)
            print(cm,'\n')
            print('PPV:', format(PPV, '.2f'),'\n')
        
        X_ext_2['Receptor'] = ext_2_receptors
        X_ext_2['hits_actual'] = ext_2_hits_actual
        X_ext_2['Score Type'] = ext_2_score_types
        X_ext_2['subset'] = ext_2_subsets
        if 'quality' in init_ext_df:
            X_ext_2['quality'] = y_ext_2
        
        X_ext_2['quality_pred'] = y_pred
        if 'quality' in init_ext_df:
            X_ext_2.to_csv('results/'+subset+'/2cluster_results.csv')
        
        if 'quality' not in init_ext_df:
            print(X_ext_2.loc[X_ext_2['quality_pred'] == 1], '\n')
            ph4_preds = X_ext_2.loc[X_ext_2['quality_pred'] == 1]
            ph4_preds.to_csv('2cluster_ph4_preds.csv')
        
    print('3 cluster model\n')
    print('---------------\n')
    if len(X_ext_3) == 0:
        print('No cluster 3 data.\n')
    else:
        y_pred = (sgdc3.predict(X_ext_3))
        if 'quality' in init_ext_df:
            confmat = confusion_matrix(y_ext_3, y_pred, labels=[0,1])
            confmat = confusion_matrix(y_ext_3, y_pred, labels=[0,1]).ravel()
            FP = (confmat[1])
            TP = (confmat[3])
            PPV = (TP / (TP + FP))
            cm = pd.crosstab(y_ext_3, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=False)
            print(cm,'\n')
            print('PPV:', format(PPV, '.2f'),'\n')
        
        X_ext_3['Receptor'] = ext_3_receptors
        X_ext_3['hits_actual'] = ext_3_hits_actual
        X_ext_3['Score Type'] = ext_3_score_types
        X_ext_3['subset'] = ext_3_subsets
        if 'quality' in init_ext_df:
            X_ext_3['quality'] = y_ext_3
        
        X_ext_3['quality_pred'] = y_pred
        
        if 'quality' in init_ext_df:
            X_ext_3.to_csv('results/'+subset+'/3cluster_results.csv')
        
        if 'quality' not in init_ext_df:
            print(X_ext_3.loc[X_ext_3['quality_pred'] == 1], '\n')
            ph4_preds = X_ext_3.loc[X_ext_3['quality_pred'] == 1]
            ph4_preds.to_csv('3cluster_ph4_preds.csv')

In [133]:
classify_ext_data('..\..\data\score_based_alldata_binary.csv', 'all')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 85 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted   0   1
Actual           
0          16   7
1          26  26 

PPV: 0.79 

2 cluster model

---------------

Predicted   0   1
Actual           
0          66  86
1          15  18 

PPV: 0.17 

3 cluster model

---------------

No cluster 3 data.



In [134]:
classify_ext_data('..\..\data\score_based_moefrags_data_binary.csv', 'moe')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 16 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          5  3
1          5  7 

PPV: 0.70 

2 cluster model

---------------

Predicted   0   1
Actual           
0          11  17
1           2   2 

PPV: 0.11 

3 cluster model

---------------

No cluster 3 data.



In [135]:
classify_ext_data('..\..\data\score_based_efdata_binary.csv', 'ef')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 20 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          2  1
1          5  4 

PPV: 0.80 

2 cluster model

---------------

Predicted   0   1
Actual           
0          17  12
1           4   7 

PPV: 0.37 

3 cluster model

---------------

No cluster 3 data.



In [136]:
classify_ext_data('..\..\data\score_based_ghdata_binary.csv', 'gh')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 21 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          5  1
1          9  6 

PPV: 0.86 

2 cluster model

---------------

Predicted  0   1
Actual          
0          9  16
1          2   4 

PPV: 0.20 

3 cluster model

---------------

No cluster 3 data.



In [137]:
classify_ext_data('..\..\data\score_based_recefdata_binary.csv', 'rec_ef')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 13 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          1  0
1          4  4 

PPV: 1.00 

2 cluster model

---------------

Predicted   0   1
Actual           
0          20  18
1           1   4 

PPV: 0.18 

3 cluster model

---------------

No cluster 3 data.



In [138]:
classify_ext_data('..\..\data\score_based_recghdata_binary.csv', 'rec_gh')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 15 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          2  3
1          4  4 

PPV: 0.57 

2 cluster model

---------------

Predicted   0   1
Actual           
0          10  22
1           6   1 

PPV: 0.04 

3 cluster model

---------------

No cluster 3 data.



In [139]:
classify_ext_data('..\..\data\hm_score_based_alldata_binary.csv', 'hm_all')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 58 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted   0   1
Actual           
0          10   1
1          27  10 

PPV: 0.91 

2 cluster model

---------------

Predicted   0    1
Actual            
0          73  118
1          12    9 

PPV: 0.07 

3 cluster model

---------------

No cluster 3 data.



In [140]:
classify_ext_data('..\..\data\hm_score_based_moefrags_data_binary.csv', 'hm_moe')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 9 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          3  1
1          4  1 

PPV: 0.50 

2 cluster model

---------------

Predicted   0   1
Actual           
0          16  23
1           2   2 

PPV: 0.08 

3 cluster model

---------------

No cluster 3 data.



In [141]:
classify_ext_data('..\..\data\hm_score_based_efdata_binary.csv', 'hm_ef')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 11 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          3  0
1          5  2 

PPV: 1.00 

2 cluster model

---------------

Predicted   0   1
Actual           
0          14  24
1           2   2 

PPV: 0.08 

3 cluster model

---------------

No cluster 3 data.



In [142]:
classify_ext_data('..\..\data\hm_score_based_efdata_binary.csv', 'hm_gh')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 11 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          3  0
1          5  2 

PPV: 1.00 

2 cluster model

---------------

Predicted   0   1
Actual           
0          14  24
1           2   2 

PPV: 0.08 

3 cluster model

---------------

No cluster 3 data.



In [143]:
classify_ext_data('..\..\data\hm_score_based_recefdata_binary.csv', 'hm_rec_ef')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 15 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

Predicted  0  1
Actual         
0          2  0
1          6  4 

PPV: 1.00 

2 cluster model

---------------

Predicted   0   1
Actual           
0          13  22
1           4   1 

PPV: 0.04 

3 cluster model

---------------

No cluster 3 data.



In [144]:
classify_ext_data('..\..\data\hm_score_based_recghdata_binary.csv', 'hm_rec_gh')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop', 'aniacc_prop'] 

score based Q ph4s: 12 

0 cluster model

---------------

Predicted  0  1
Actual         
0          0  1
1          2  5 

PPV: 0.83 

1 cluster model

---------------

No cluster 1 data.

2 cluster model

---------------

Predicted   0   1
Actual           
0          15  24
1           2   3 

PPV: 0.11 

3 cluster model

---------------

No cluster 3 data.



In [146]:
classify_ext_data('..\..\data\gpr101_data_binary_5feats.csv')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop'] 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

     s_score      Hits  max_feat  avg_feat  max_centr  min_centr  avg_centr  \
21 -0.111182 -0.696069  1.111583  0.981812   0.880114   0.461244   1.079794   
29 -0.067695 -0.706324  1.321852  0.493215   1.120982   0.899270   1.359525   
37  0.030912 -0.716580  1.321852  1.108692   1.120982   0.899270   1.204100   

    features  all_same  hyd_prop  don_prop  catdon_prop  hydaro_prop  \
21       0.0 -0.538816 -0.226463 -0.816497     0.561548     1.538968   
29       0.0 -0.538816 -0.703229 -0.816497     0.942259     1.538968   
37       0.0 -0.538816 -0.703229 -0.816497     0.942259    -0.512989   

    aniacc_prop  clusters              Receptor  hits_actual Score Type  \
21    -0.267261  -1.36277  GPR101_5ZKC_inactive         

In [147]:
classify_ext_data('..\..\data\gpr101_data_binary_6feats.csv')

Predictors: ['s_score', 'Hits', 'max_feat', 'avg_feat', 'max_centr', 'min_centr', 'avg_centr', 'features', 'all_same', 'hyd_prop', 'don_prop', 'catdon_prop', 'hydaro_prop'] 

0 cluster model

---------------

No cluster 0 data.

1 cluster model

---------------

No cluster 1 data.

2 cluster model

---------------

     s_score      Hits  max_feat  avg_feat  max_centr  min_centr  avg_centr  \
0   0.496935 -0.380754  0.618669  0.561454   0.128391  -0.660837  -0.306940   
4   0.506753  0.287235  0.618669  0.500210   0.128391  -0.660837   0.024740   
6   0.408967  0.153637  0.396937  0.031883  -0.685094  -0.660837  -0.896230   
8   0.399251 -0.380754  0.618669  0.561454   0.128391  -0.660837  -0.306940   
12  0.194259 -0.113558  0.469649  0.197502  -0.288910  -1.178886  -0.717330   
14  0.093021  0.020040  0.366329  0.024257  -0.288910  -1.061633  -0.783969   
16  0.801647 -0.514352  0.246887  0.409493  -0.223056  -1.097427  -0.863452   
17 -0.026903 -0.514352 -0.282375 -0.001355   0.0731

In [29]:
#save models
#import joblib
#from joblib import dump, load

#joblib.dump(sgdc0 , 'model_sgdc0')
#joblib.dump(sgdc0 , 'model_sgdc1')
#joblib.dump(sgdc0 , 'model_sgdc2')
#joblib.dump(sgdc0 , 'model_sgdc3')